### Regression notebook for Wadhwani AI competition

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import os
import gc
import random
from glob import glob
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
import warnings
import pickle
import json
import re
import time
import sys
from requests import get
import multiprocessing
import joblib
import torch
from torch.utils.data import Dataset, DataLoader
import transformers
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import timm
from sklearn.preprocessing import minmax_scale
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2,torchvision
from ipyexperiments.ipyexperiments import IPyExperimentsPytorch
from timm.optim.optim_factory import create_optimizer_v2
from timm import utils
from fastprogress.fastprogress import format_time
from fastai.vision.all import *
from sklearn.multioutput import MultiOutputRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
class CFG:
    seed = 46
    n_splits = 5
    debug = False
    MODEL = 'lgbm'
    
random.seed(CFG.seed)
os.environ["PYTHONHASHSEED"] = str(CFG.seed)
np.random.seed(CFG.seed)
plt.rcParams["font.size"] = 13
warnings.filterwarnings('ignore')

In [2]:
set_seed(CFG.seed)

In [393]:
DIR = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/'
IMG_PATH = '///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/images'
submit = pd.read_csv(os.path.join(DIR,'SampleSubmission.csv'))
train = pd.read_csv(os.path.join(DIR,'Train.csv'))
test_df = pd.read_csv(os.path.join(DIR,'Test.csv'))

VERSION = "V001"
MODEL_FOLDER = Path(f"///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/LGBM/{VERSION}/")
os.makedirs(MODEL_FOLDER,exist_ok=True)
KERNEL_TYPE = f'{CFG.MODEL}_{VERSION}'

print(MODEL_FOLDER)
print(KERNEL_TYPE)

/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/LGBM/V001
lgbm_V001


In [4]:
def make_train_dataset():
    train_new_pbw = pd.DataFrame()
    train_new_pbw['image_id_worm']= train['image_id_worm'].unique()
    train_new_pbw = pd.merge(train_new_pbw,train[train['worm_type']=='pbw'].reset_index(drop=True),on='image_id_worm',how='left')
    train_new_pbw['worm_type'] = 'pbw'
    train_new_pbw.fillna(0,inplace=True)
    
    train_new_abw = pd.DataFrame()
    train_new_abw['image_id_worm']= train['image_id_worm'].unique()
    train_new_abw = pd.merge(train_new_abw,train[train['worm_type']=='abw'].reset_index(drop=True),on='image_id_worm',how='left')
    train_new_abw['worm_type'] = 'abw'
    train_new_abw.fillna(0,inplace=True)
    
    train_out = pd.concat([train_new_pbw,train_new_abw],0).reset_index(drop=True)
    
    assert len(train_out) == train['image_id_worm'].nunique()*2
    train_out = pd.pivot(train_out,'image_id_worm','worm_type','number_of_worms').reset_index()
    train_out[['abw','pbw']] = train_out[['abw','pbw']].astype(int)
    
    labels = [f'{i}' for i in range(10)]
    train_out['abw_bins'] = pd.cut(train_out['abw'],10,labels=labels)
    train_out['pbw_bins'] = pd.cut(train_out['pbw'],10,labels=labels)
    train_out['consol_bins'] = train_out['abw_bins'].astype(str)+'_'+train_out['pbw_bins'].astype(str)
    
#     train_out = train_out[['image_id_worm','abw','pbw','abw_bins','pbw_bins','consol_bins']]
    return train_out

train_new = make_train_dataset()

In [5]:
train_new.head(1)

worm_type,image_id_worm,abw,pbw,abw_bins,pbw_bins,consol_bins
0,id_0002ea6f15c7fa6f4c221783.jpg,0,51,0,0,0_0


In [6]:
train_new.shape

(9737, 6)

### Get kfolds

In [7]:
train_files = []
val_files = []
fold = []

for folds in [0,1,2,3,4]:
    files = list(pd.read_csv(f'///mnt/c/Personal/Competitions/Zindi/Wadhwani AI/data/splits/fold{folds}.txt',header=None)[0].apply(lambda x:x.split("/")[-1]))
    fold.append([folds]*len(files))
    train_files.append(files)

train_files= ([item for sublist in train_files for item in sublist])
fold= ([item for sublist in fold for item in sublist])
fold_dict = dict(zip(train_files,fold))

In [8]:
train_new['fold'] = train_new['image_id_worm'].map(fold_dict)
train_new = train_new.loc[~train_new['fold'].isna()].reset_index(drop=True)
# train_new['fold'].fillna(5,inplace=True)

In [9]:
train_new.shape

(7079, 7)

In [10]:
### Load YOLO preds

In [11]:
with open('/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/yolov5s6_exp001_val.pickle', 'rb') as handle:
    yValFeats = pickle.load(handle)
    
with open('/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/yolov5s6_exp002_val.pickle', 'rb') as handle:
    yValFeats1 = pickle.load(handle)
    
with open('/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/yolov5s6_exp001_train.pickle', 'rb') as handle:
    yTrFeats = pickle.load(handle)
    
with open('/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/yolov5s6_exp002_train.pickle', 'rb') as handle:
    yTrFeats1 = pickle.load(handle)
    
with open('/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/yolov5s6_exp001_test.pickle', 'rb') as handle:
    yTestFeats = pickle.load(handle)
    
with open('/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/yolov5s6_exp002_test.pickle', 'rb') as handle:
    yTestFeats1 = pickle.load(handle)

In [12]:
len(yValFeats.keys())

1418

In [16]:
def getYoloDF(featDict,yolonew=False):
    d = {k[0] : v.cpu().numpy() for k, v in featDict.items()}
    out = pd.DataFrame()
    i = 0
    for k,v in tqdm(d.items()):
        v = v.flatten()
        out.loc[i,'id'] = k
        for j in range(v.shape[0]):
            j_n = j
            if yolonew:
                j_n = f'{j}_new'
            out.loc[i,f'yolo_feat_{j_n}'] = v[j]
        i += 1
    return out

yValFeatsDF = getYoloDF(yValFeats)
yValFeats1DF = getYoloDF(yValFeats1,yolonew=True)
yTrFeatsDF = getYoloDF(yTrFeats)
yTrFeats1DF = getYoloDF(yTrFeats1,yolonew=True)
yTestFeatsDF = getYoloDF(yTestFeats)
yTestFeats1DF = getYoloDF(yTestFeats1,yolonew=True)

  0%|          | 0/1418 [00:00<?, ?it/s]

  0%|          | 0/1418 [00:00<?, ?it/s]

  0%|          | 0/5661 [00:00<?, ?it/s]

  0%|          | 0/5661 [00:00<?, ?it/s]

  0%|          | 0/2803 [00:00<?, ?it/s]

  0%|          | 0/2803 [00:00<?, ?it/s]

In [17]:
print(yValFeatsDF.shape,yValFeats1DF.shape)

(1418, 513) (1418, 513)


In [18]:
print(yTrFeatsDF.shape,yTrFeats1DF.shape)

(5661, 513) (5661, 513)


In [19]:
print(yTestFeatsDF.shape,yTestFeats1DF.shape)

(2803, 513) (2803, 513)


### Reg feats

In [20]:
with open(f'/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/train_reg.pickle', 'rb') as handle:
    RegTrainFeats = pickle.load(handle)
    
with open(f'/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/val_reg.pickle', 'rb') as handle:
    RegValFeats = pickle.load(handle)
    
with open(f'/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/YOLO_Features/test_reg.pickle', 'rb') as handle:
    RegTestFeats = pickle.load(handle)


In [26]:
RegTrainFeatsDF = pd.DataFrame.from_dict(RegTrainFeats,orient='index')
RegTrainFeatsDF.reset_index(inplace=True)
RegTrainFeatsDF.columns = ['id','Reg_feat_1','Reg_feat_2']

RegValFeatsDF = pd.DataFrame.from_dict(RegValFeats,orient='index')
RegValFeatsDF.reset_index(inplace=True)
RegValFeatsDF.columns = ['id','Reg_feat_1','Reg_feat_2']

RegTestFeatsDF = pd.DataFrame.from_dict(RegTestFeats,orient='index')
RegTestFeatsDF.reset_index(inplace=True)
RegTestFeatsDF.columns = ['id','Reg_feat_1','Reg_feat_2']
print(RegTrainFeatsDF.shape,RegValFeatsDF.shape,RegTestFeatsDF.shape)

(5661, 3) (1418, 3) (2803, 3)


In [27]:
RegTrainFeatsDF.head()

,id,Reg_feat_1,Reg_feat_2
0,id_0002ea6f15c7fa6f4c221783.jpg,0.000000,44.477036
1,id_00093f2c76f6488737325859.jpg,0.000000,6.624971
2,id_001149c0de05ea4941d966e0.jpg,4.910912,0.000000
3,id_002b8a0e15968eaf2e61f4c7.jpg,0.000000,7.970926
4,id_003ad8831eae2f0117f5aaf2.jpg,0.000000,4.219037


### Consol DF

In [315]:
consol_train_yolo = pd.merge(pd.concat([yTrFeatsDF,yTrFeats1DF.iloc[:,1:]],1),RegTrainFeatsDF,on='id',how='left')
consol_train_yolo.head()

,id,yolo_feat_0,yolo_feat_1,yolo_feat_2,yolo_feat_3,yolo_feat_4,yolo_feat_5,yolo_feat_6,yolo_feat_7,yolo_feat_8,...,yolo_feat_504_new,yolo_feat_505_new,yolo_feat_506_new,yolo_feat_507_new,yolo_feat_508_new,yolo_feat_509_new,yolo_feat_510_new,yolo_feat_511_new,Reg_feat_1,Reg_feat_2
0,id_0002ea6f15c7fa6f4c221783.jpg,0.324414,-0.076526,0.149614,0.481668,0.402245,0.376715,1.184626,0.151707,0.394637,...,0.405878,0.554902,0.516051,-0.106231,-0.163378,0.355666,0.129848,1.435313,0.000000,44.477036
1,id_00093f2c76f6488737325859.jpg,0.445402,-0.039216,0.312715,0.754545,0.646009,0.516677,1.469848,0.212772,0.465128,...,0.669820,0.617772,0.714937,-0.175738,-0.220584,0.438421,0.080329,0.989508,0.000000,6.624971
2,id_001149c0de05ea4941d966e0.jpg,0.343728,-0.112397,0.280331,0.686784,0.555225,0.338934,1.410484,0.065411,0.189863,...,0.396779,0.435038,0.527400,0.033634,-0.144069,0.411828,-0.004731,1.047535,4.910912,0.000000
3,id_002b8a0e15968eaf2e61f4c7.jpg,0.353630,-0.033530,0.306705,0.566820,0.339751,0.579327,1.171099,0.222281,0.565533,...,0.682715,0.599436,0.605213,0.093984,-0.083339,0.216596,0.212982,1.539990,0.000000,7.970926
4,id_003ad8831eae2f0117f5aaf2.jpg,0.374285,-0.068414,0.370380,0.732315,0.498809,0.519107,1.391410,0.117057,0.325781,...,0.412729,0.485265,0.438969,0.026372,-0.165158,0.432129,0.157867,1.199102,0.000000,4.219037


In [316]:
consol_val_yolo = pd.merge(pd.concat([yValFeatsDF,yValFeats1DF.iloc[:,1:]],1),RegValFeatsDF,on='id',how='left')
consol_val_yolo.head()

,id,yolo_feat_0,yolo_feat_1,yolo_feat_2,yolo_feat_3,yolo_feat_4,yolo_feat_5,yolo_feat_6,yolo_feat_7,yolo_feat_8,...,yolo_feat_504_new,yolo_feat_505_new,yolo_feat_506_new,yolo_feat_507_new,yolo_feat_508_new,yolo_feat_509_new,yolo_feat_510_new,yolo_feat_511_new,Reg_feat_1,Reg_feat_2
0,id_0005ef295aafe6acc63587db.jpg,0.110916,-0.154880,-0.031581,0.132209,0.196372,0.222387,0.722178,0.021244,0.291291,...,0.491610,0.293717,0.394240,0.524277,0.120745,0.279314,0.201766,1.288328,0.0,7.068529
1,id_000b2e6c437c643f25d4a6c3.jpg,0.323326,-0.031811,0.226457,0.538890,0.427992,0.448398,1.027566,0.200125,0.513489,...,0.510272,0.565921,0.589351,-0.129711,-0.179242,0.385599,0.079282,1.076321,0.0,75.067154
2,id_000c2040da4b05816cefbb96.jpg,0.462650,0.077881,0.472382,0.875376,0.566185,0.775293,1.223518,0.344005,0.705188,...,0.823244,0.548207,0.651464,-0.118966,-0.202680,0.576581,-0.011700,0.607950,0.0,0.000000
3,id_00b6b77332b132dbb58a7dc4.jpg,0.189851,-0.142968,0.081666,0.352681,0.394507,0.189989,1.048600,0.023760,0.143370,...,0.453135,0.257602,0.470692,0.239762,-0.056286,0.291171,-0.001095,0.784903,0.0,6.683640
4,id_00d5771ea3336b8bec3efadc.jpg,0.261536,-0.067871,0.192256,0.428570,0.247744,0.443815,0.999553,0.133940,0.459964,...,0.722533,0.518566,0.631798,0.071694,-0.055875,0.336665,0.171261,1.232476,0.0,3.404002


In [317]:
consol_test_yolo = pd.merge(pd.concat([yTestFeatsDF,yTestFeats1DF.iloc[:,1:]],1),RegTestFeatsDF,on='id',how='left')
consol_test_yolo.head()

,id,yolo_feat_0,yolo_feat_1,yolo_feat_2,yolo_feat_3,yolo_feat_4,yolo_feat_5,yolo_feat_6,yolo_feat_7,yolo_feat_8,...,yolo_feat_504_new,yolo_feat_505_new,yolo_feat_506_new,yolo_feat_507_new,yolo_feat_508_new,yolo_feat_509_new,yolo_feat_510_new,yolo_feat_511_new,Reg_feat_1,Reg_feat_2
0,id_00332970f80fa9a47a39516d.jpg,0.438913,-0.048683,0.239698,0.685954,0.711913,0.535881,1.455314,0.228507,0.513024,...,0.702751,0.491164,0.627846,0.092285,-0.177102,0.343984,0.048436,0.871077,3.618258,0.000000
1,id_0035981bc3ae42eb5b57a317.jpg,0.476342,-0.041817,0.246218,0.697897,0.639696,0.511753,1.550630,0.235087,0.486410,...,0.531950,0.722890,0.740556,-0.132606,-0.186134,0.330609,0.200698,1.591264,0.000000,20.003084
2,id_005102f664b820f778291dee.jpg,0.346716,-0.099539,0.312692,0.680482,0.567164,0.395383,1.360361,0.073132,0.233578,...,0.355814,0.461989,0.457790,0.086513,-0.123350,0.373320,0.011149,1.061191,9.088399,0.000000
3,id_0066456f5fb2cd858c69ab39.jpg,0.383099,-0.076594,0.628910,1.007551,0.553743,0.555077,1.467012,0.071154,0.236531,...,0.678455,0.619724,0.815335,-0.170632,-0.214970,0.475475,0.016497,0.992888,4.259914,0.000000
4,id_007159c1fa015ba6f394deeb.jpg,0.361320,-0.029009,0.380299,0.657042,0.380595,0.654411,1.189501,0.213645,0.599163,...,0.817781,0.632519,0.877259,-0.019525,-0.177327,0.372282,0.105089,1.117604,0.000000,0.000000


In [318]:
train_new.head()

worm_type,image_id_worm,abw,pbw,abw_bins,pbw_bins,consol_bins,fold
0,id_0002ea6f15c7fa6f4c221783.jpg,0,51,0,0,0_0,1.0
1,id_0005ef295aafe6acc63587db.jpg,0,8,0,0,0_0,0.0
2,id_00093f2c76f6488737325859.jpg,0,12,0,0,0_0,2.0
3,id_000b2e6c437c643f25d4a6c3.jpg,0,87,0,1,0_1,0.0
4,id_000c2040da4b05816cefbb96.jpg,0,0,0,0,0_0,0.0


In [319]:
temp = train_new[['image_id_worm','abw','pbw']]
temp.columns = ['id','abw','pbw']

consol_train_yolo = pd.merge(consol_train_yolo,temp,on='id',how='left')
consol_train_yolo.head()

,id,yolo_feat_0,yolo_feat_1,yolo_feat_2,yolo_feat_3,yolo_feat_4,yolo_feat_5,yolo_feat_6,yolo_feat_7,yolo_feat_8,...,yolo_feat_506_new,yolo_feat_507_new,yolo_feat_508_new,yolo_feat_509_new,yolo_feat_510_new,yolo_feat_511_new,Reg_feat_1,Reg_feat_2,abw,pbw
0,id_0002ea6f15c7fa6f4c221783.jpg,0.324414,-0.076526,0.149614,0.481668,0.402245,0.376715,1.184626,0.151707,0.394637,...,0.516051,-0.106231,-0.163378,0.355666,0.129848,1.435313,0.000000,44.477036,0,51
1,id_00093f2c76f6488737325859.jpg,0.445402,-0.039216,0.312715,0.754545,0.646009,0.516677,1.469848,0.212772,0.465128,...,0.714937,-0.175738,-0.220584,0.438421,0.080329,0.989508,0.000000,6.624971,0,12
2,id_001149c0de05ea4941d966e0.jpg,0.343728,-0.112397,0.280331,0.686784,0.555225,0.338934,1.410484,0.065411,0.189863,...,0.527400,0.033634,-0.144069,0.411828,-0.004731,1.047535,4.910912,0.000000,5,0
3,id_002b8a0e15968eaf2e61f4c7.jpg,0.353630,-0.033530,0.306705,0.566820,0.339751,0.579327,1.171099,0.222281,0.565533,...,0.605213,0.093984,-0.083339,0.216596,0.212982,1.539990,0.000000,7.970926,0,7
4,id_003ad8831eae2f0117f5aaf2.jpg,0.374285,-0.068414,0.370380,0.732315,0.498809,0.519107,1.391410,0.117057,0.325781,...,0.438969,0.026372,-0.165158,0.432129,0.157867,1.199102,0.000000,4.219037,0,8


In [320]:
consol_val_yolo = pd.merge(consol_val_yolo,temp,on='id',how='left')
consol_val_yolo.head()

,id,yolo_feat_0,yolo_feat_1,yolo_feat_2,yolo_feat_3,yolo_feat_4,yolo_feat_5,yolo_feat_6,yolo_feat_7,yolo_feat_8,...,yolo_feat_506_new,yolo_feat_507_new,yolo_feat_508_new,yolo_feat_509_new,yolo_feat_510_new,yolo_feat_511_new,Reg_feat_1,Reg_feat_2,abw,pbw
0,id_0005ef295aafe6acc63587db.jpg,0.110916,-0.154880,-0.031581,0.132209,0.196372,0.222387,0.722178,0.021244,0.291291,...,0.394240,0.524277,0.120745,0.279314,0.201766,1.288328,0.0,7.068529,0,8
1,id_000b2e6c437c643f25d4a6c3.jpg,0.323326,-0.031811,0.226457,0.538890,0.427992,0.448398,1.027566,0.200125,0.513489,...,0.589351,-0.129711,-0.179242,0.385599,0.079282,1.076321,0.0,75.067154,0,87
2,id_000c2040da4b05816cefbb96.jpg,0.462650,0.077881,0.472382,0.875376,0.566185,0.775293,1.223518,0.344005,0.705188,...,0.651464,-0.118966,-0.202680,0.576581,-0.011700,0.607950,0.0,0.000000,0,0
3,id_00b6b77332b132dbb58a7dc4.jpg,0.189851,-0.142968,0.081666,0.352681,0.394507,0.189989,1.048600,0.023760,0.143370,...,0.470692,0.239762,-0.056286,0.291171,-0.001095,0.784903,0.0,6.683640,0,7
4,id_00d5771ea3336b8bec3efadc.jpg,0.261536,-0.067871,0.192256,0.428570,0.247744,0.443815,0.999553,0.133940,0.459964,...,0.631798,0.071694,-0.055875,0.336665,0.171261,1.232476,0.0,3.404002,0,3


### LGBM

In [321]:
train_new.shape,consol_train_yolo.shape,consol_val_yolo.shape,consol_test_yolo.shape

((7079, 7), (5661, 1029), (1418, 1029), (2803, 1027))

In [322]:
consol_test_yolo.iloc[:,1:].head()

,yolo_feat_0,yolo_feat_1,yolo_feat_2,yolo_feat_3,yolo_feat_4,yolo_feat_5,yolo_feat_6,yolo_feat_7,yolo_feat_8,yolo_feat_9,...,yolo_feat_504_new,yolo_feat_505_new,yolo_feat_506_new,yolo_feat_507_new,yolo_feat_508_new,yolo_feat_509_new,yolo_feat_510_new,yolo_feat_511_new,Reg_feat_1,Reg_feat_2
0,0.438913,-0.048683,0.239698,0.685954,0.711913,0.535881,1.455314,0.228507,0.513024,-0.103025,...,0.702751,0.491164,0.627846,0.092285,-0.177102,0.343984,0.048436,0.871077,3.618258,0.000000
1,0.476342,-0.041817,0.246218,0.697897,0.639696,0.511753,1.550630,0.235087,0.486410,-0.073653,...,0.531950,0.722890,0.740556,-0.132606,-0.186134,0.330609,0.200698,1.591264,0.000000,20.003084
2,0.346716,-0.099539,0.312692,0.680482,0.567164,0.395383,1.360361,0.073132,0.233578,0.094824,...,0.355814,0.461989,0.457790,0.086513,-0.123350,0.373320,0.011149,1.061191,9.088399,0.000000
3,0.383099,-0.076594,0.628910,1.007551,0.553743,0.555077,1.467012,0.071154,0.236531,-0.102313,...,0.678455,0.619724,0.815335,-0.170632,-0.214970,0.475475,0.016497,0.992888,4.259914,0.000000
4,0.361320,-0.029009,0.380299,0.657042,0.380595,0.654411,1.189501,0.213645,0.599163,-0.079315,...,0.817781,0.632519,0.877259,-0.019525,-0.177327,0.372282,0.105089,1.117604,0.000000,0.000000


In [323]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(consol_val_yolo[['abw','pbw']],np.round(consol_val_yolo[['Reg_feat_1','Reg_feat_2']]))

2.244358251057828

In [329]:
def training_loop():
    out = pd.DataFrame()
    pred_ = pd.DataFrame()
   
    X_tr = consol_train_yolo.iloc[:,1:-2]
    Y_tr = consol_train_yolo.iloc[:,-2:]

    X_val = consol_val_yolo.iloc[:,1:-2]
    Y_val = consol_val_yolo.iloc[:,-2:]

    X_test = consol_test_yolo.iloc[:,1:]
        
    for i in range(2):
        nest = 150
        model = LGBMRegressor(boosting_type='gbdt',
#                               objective='regression_l2',
                              n_estimators=nest,
#                               learning_rate=0.02,
                              n_jobs=-1,
                              random_state=42,
#                               min_split_gain = 0.0001,
#                               min_child_samples = 25,
#                               reg_alpha=10,
                              num_leaves=41,                    
                              metric='mae')
#                              subsample=1,)
        model.fit(X_tr, Y_tr.iloc[:,i], 
                  eval_set=[(X_tr, Y_tr.iloc[:,i]),(X_val,Y_val.iloc[:,i])],
                 eval_names = ['train','val'])    
        
        p = np.round(model.predict(X_val).clip(0,600))
        pred = np.round(model.predict(X_test).clip(0,600))
        
        out = pd.concat([out,pd.DataFrame(pred)],1)
        pred_ = pd.concat([pred_,pd.DataFrame(p)],1)
    
    pred_.columns = ['abw','pbw']
    
    print(f'mean_absolute_error:{mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))}')
    return out,pred_,mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))

In [330]:
testPred,Valpred_,mae = training_loop()

[1]	train's l1: 1.83227	val's l1: 1.78494
[2]	train's l1: 1.6548	val's l1: 1.61988
[3]	train's l1: 1.49576	val's l1: 1.47138
[4]	train's l1: 1.35409	val's l1: 1.34132
[5]	train's l1: 1.2272	val's l1: 1.22616
[6]	train's l1: 1.1131	val's l1: 1.12204
[7]	train's l1: 1.01106	val's l1: 1.02894
[8]	train's l1: 0.919065	val's l1: 0.944547
[9]	train's l1: 0.835839	val's l1: 0.869648
[10]	train's l1: 0.761355	val's l1: 0.802377
[11]	train's l1: 0.694193	val's l1: 0.741243
[12]	train's l1: 0.633945	val's l1: 0.686335
[13]	train's l1: 0.579783	val's l1: 0.636818
[14]	train's l1: 0.531419	val's l1: 0.593278
[15]	train's l1: 0.488103	val's l1: 0.553711
[16]	train's l1: 0.448595	val's l1: 0.51821
[17]	train's l1: 0.412846	val's l1: 0.486906
[18]	train's l1: 0.381198	val's l1: 0.45861
[19]	train's l1: 0.35296	val's l1: 0.434314
[20]	train's l1: 0.327186	val's l1: 0.412064
[21]	train's l1: 0.304378	val's l1: 0.391958
[22]	train's l1: 0.28394	val's l1: 0.374759
[23]	train's l1: 0.265626	val's l1: 0.35

[44]	train's l1: 1.81141	val's l1: 4.45068
[45]	train's l1: 1.78732	val's l1: 4.44332
[46]	train's l1: 1.76108	val's l1: 4.43998
[47]	train's l1: 1.73977	val's l1: 4.43676
[48]	train's l1: 1.72134	val's l1: 4.43187
[49]	train's l1: 1.7003	val's l1: 4.43355
[50]	train's l1: 1.6878	val's l1: 4.43254
[51]	train's l1: 1.67511	val's l1: 4.43274
[52]	train's l1: 1.65677	val's l1: 4.4328
[53]	train's l1: 1.6446	val's l1: 4.43306
[54]	train's l1: 1.63214	val's l1: 4.43086
[55]	train's l1: 1.61576	val's l1: 4.42718
[56]	train's l1: 1.6025	val's l1: 4.42932
[57]	train's l1: 1.58934	val's l1: 4.43936
[58]	train's l1: 1.57454	val's l1: 4.44151
[59]	train's l1: 1.56223	val's l1: 4.44374
[60]	train's l1: 1.54879	val's l1: 4.44429
[61]	train's l1: 1.53954	val's l1: 4.44591
[62]	train's l1: 1.52594	val's l1: 4.43814
[63]	train's l1: 1.51414	val's l1: 4.43896
[64]	train's l1: 1.5044	val's l1: 4.43962
[65]	train's l1: 1.49344	val's l1: 4.44142
[66]	train's l1: 1.48429	val's l1: 4.43805
[67]	train's l1: 

In [336]:
def training_loop_new():
    out = pd.DataFrame()
    pred_ = pd.DataFrame()
   
    X_tr = consol_train_yolo.iloc[:,1:-2]
    Y_tr = consol_train_yolo.iloc[:,-2:]

    X_val = consol_val_yolo.iloc[:,1:-2]
    Y_val = consol_val_yolo.iloc[:,-2:]

    X_test = consol_test_yolo.iloc[:,1:]
        
    for i in range(2):
        nest = 150
        model = LGBMRegressor(boosting_type='gbdt',
                              n_estimators=nest,
                              n_jobs=-1,
                              random_state=42,
                              num_leaves=41,                    
                              metric='mae')
        model.fit(X_tr, Y_tr.iloc[:,i], 
                  eval_set=[(X_tr, Y_tr.iloc[:,i]),(X_val,Y_val.iloc[:,i])],
                 eval_names = ['train','val'])    

        
        trP = np.round(model.predict(X_val).clip(0,600))
        valP = np.round(model.predict(X_val).clip(0,600))
        testP = np.round(model.predict(X_test).clip(0,600))
        
        X_tr_new = pd.concat([X_tr,pd.DataFrame(trP)],1)
        X_val_new = pd.concat([X_val,pd.DataFrame(valP)],1)
        X_test_new = pd.concat([X_tr,pd.DataFrame(testP)],1)
        
        model = LGBMRegressor(boosting_type='gbdt',
                              n_estimators=nest,
                              n_jobs=-1,
                              random_state=42,
                              num_leaves=41,                    
                              metric='mae')
        model.fit(X_tr_new, Y_tr.iloc[:,i], 
                  eval_set=[(X_tr_new, Y_tr.iloc[:,i]),(X_val_new,Y_val.iloc[:,i])],
                 eval_names = ['train','val'])
        p = np.round(model.predict(X_val_new).clip(0,600))
        pred = np.round(model.predict(X_test_new).clip(0,600))
        out = pd.concat([out,pd.DataFrame(pred)],1)
        pred_ = pd.concat([pred_,pd.DataFrame(p)],1)
        
    pred_.columns = ['abw','pbw']
    print(f'mean_absolute_error:{mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))}')
    return out,pred_,mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))

In [337]:
training_loop_new()

[1]	train's l1: 1.83227	val's l1: 1.78494
[2]	train's l1: 1.6548	val's l1: 1.61988
[3]	train's l1: 1.49576	val's l1: 1.47138
[4]	train's l1: 1.35409	val's l1: 1.34132
[5]	train's l1: 1.2272	val's l1: 1.22616
[6]	train's l1: 1.1131	val's l1: 1.12204
[7]	train's l1: 1.01106	val's l1: 1.02894
[8]	train's l1: 0.919065	val's l1: 0.944547
[9]	train's l1: 0.835839	val's l1: 0.869648
[10]	train's l1: 0.761355	val's l1: 0.802377
[11]	train's l1: 0.694193	val's l1: 0.741243
[12]	train's l1: 0.633945	val's l1: 0.686335
[13]	train's l1: 0.579783	val's l1: 0.636818
[14]	train's l1: 0.531419	val's l1: 0.593278
[15]	train's l1: 0.488103	val's l1: 0.553711
[16]	train's l1: 0.448595	val's l1: 0.51821
[17]	train's l1: 0.412846	val's l1: 0.486906
[18]	train's l1: 0.381198	val's l1: 0.45861
[19]	train's l1: 0.35296	val's l1: 0.434314
[20]	train's l1: 0.327186	val's l1: 0.412064
[21]	train's l1: 0.304378	val's l1: 0.391958
[22]	train's l1: 0.28394	val's l1: 0.374759
[23]	train's l1: 0.265626	val's l1: 0.35

[41]	train's l1: 0.124284	val's l1: 0.256937
[42]	train's l1: 0.121736	val's l1: 0.254953
[43]	train's l1: 0.119014	val's l1: 0.253541
[44]	train's l1: 0.116678	val's l1: 0.252456
[45]	train's l1: 0.114478	val's l1: 0.251345
[46]	train's l1: 0.112511	val's l1: 0.250784
[47]	train's l1: 0.110515	val's l1: 0.249725
[48]	train's l1: 0.108998	val's l1: 0.248617
[49]	train's l1: 0.106908	val's l1: 0.247767
[50]	train's l1: 0.105367	val's l1: 0.246976
[51]	train's l1: 0.103625	val's l1: 0.246273
[52]	train's l1: 0.101812	val's l1: 0.246002
[53]	train's l1: 0.100246	val's l1: 0.245521
[54]	train's l1: 0.0987421	val's l1: 0.245689
[55]	train's l1: 0.0976611	val's l1: 0.24538
[56]	train's l1: 0.096322	val's l1: 0.2446
[57]	train's l1: 0.0953219	val's l1: 0.244828
[58]	train's l1: 0.0943801	val's l1: 0.245076
[59]	train's l1: 0.0933806	val's l1: 0.244715
[60]	train's l1: 0.0926935	val's l1: 0.244959
[61]	train's l1: 0.0919118	val's l1: 0.245507
[62]	train's l1: 0.0912323	val's l1: 0.245649
[63]	

[78]	train's l1: 1.37396	val's l1: 4.45128
[79]	train's l1: 1.36519	val's l1: 4.45405
[80]	train's l1: 1.35504	val's l1: 4.44998
[81]	train's l1: 1.34645	val's l1: 4.4468
[82]	train's l1: 1.34052	val's l1: 4.45064
[83]	train's l1: 1.33139	val's l1: 4.45061
[84]	train's l1: 1.32454	val's l1: 4.45719
[85]	train's l1: 1.3173	val's l1: 4.45806
[86]	train's l1: 1.30999	val's l1: 4.45805
[87]	train's l1: 1.30309	val's l1: 4.46235
[88]	train's l1: 1.29754	val's l1: 4.46444
[89]	train's l1: 1.29027	val's l1: 4.46395
[90]	train's l1: 1.28439	val's l1: 4.46176
[91]	train's l1: 1.27932	val's l1: 4.46272
[92]	train's l1: 1.26977	val's l1: 4.46048
[93]	train's l1: 1.2631	val's l1: 4.4616
[94]	train's l1: 1.25698	val's l1: 4.46059
[95]	train's l1: 1.24963	val's l1: 4.46188
[96]	train's l1: 1.2431	val's l1: 4.4614
[97]	train's l1: 1.23462	val's l1: 4.46029
[98]	train's l1: 1.22851	val's l1: 4.46171
[99]	train's l1: 1.22191	val's l1: 4.46357
[100]	train's l1: 1.21422	val's l1: 4.46704
[101]	train's l1

[122]	train's l1: 1.0639	val's l1: 4.47115
[123]	train's l1: 1.05804	val's l1: 4.47585
[124]	train's l1: 1.05206	val's l1: 4.47847
[125]	train's l1: 1.04623	val's l1: 4.47888
[126]	train's l1: 1.04029	val's l1: 4.47779
[127]	train's l1: 1.03337	val's l1: 4.48091
[128]	train's l1: 1.0272	val's l1: 4.48313
[129]	train's l1: 1.01961	val's l1: 4.48296
[130]	train's l1: 1.01489	val's l1: 4.4863
[131]	train's l1: 1.0107	val's l1: 4.48988
[132]	train's l1: 1.0052	val's l1: 4.48944
[133]	train's l1: 0.999603	val's l1: 4.49026
[134]	train's l1: 0.992814	val's l1: 4.48938
[135]	train's l1: 0.987824	val's l1: 4.48948
[136]	train's l1: 0.981568	val's l1: 4.49139
[137]	train's l1: 0.976572	val's l1: 4.49113
[138]	train's l1: 0.9705	val's l1: 4.48794
[139]	train's l1: 0.96488	val's l1: 4.48682
[140]	train's l1: 0.960081	val's l1: 4.48705
[141]	train's l1: 0.953293	val's l1: 4.48842
[142]	train's l1: 0.948712	val's l1: 4.49263
[143]	train's l1: 0.943842	val's l1: 4.49523
[144]	train's l1: 0.938404	va

(        0     0
 0     0.0  50.0
 1     0.0   8.0
 2     5.0   0.0
 3     0.0   7.0
 4     0.0   5.0
 ...   ...   ...
 5656  0.0  12.0
 5657  0.0  78.0
 5658  5.0   0.0
 5659  0.0  97.0
 5660  0.0  32.0
 
 [5661 rows x 2 columns],
        abw   pbw
 0      0.0   6.0
 1      0.0  75.0
 2      0.0   1.0
 3      0.0   7.0
 4      0.0   3.0
 ...    ...   ...
 1413  10.0   0.0
 1414   0.0  27.0
 1415   2.0   0.0
 1416   0.0   3.0
 1417   0.0  29.0
 
 [1418 rows x 2 columns],
 2.306064880112835)

In [367]:
from sklearn.svm import SVR

def training_loop_SVR():
    
    out = pd.DataFrame()
    pred_ = pd.DataFrame()
   
    X_tr = consol_train_yolo.iloc[:,1:-2]
    Y_tr = consol_train_yolo.iloc[:,-2:]

    X_val = consol_val_yolo.iloc[:,1:-2]
    Y_val = consol_val_yolo.iloc[:,-2:]

    X_test = consol_test_yolo.iloc[:,1:]
        
    for i in range(2):
        model = SVR(verbose=True,kernel='rbf',C=10000)
        model.fit(X_tr, Y_tr.iloc[:,i])
        p = np.round(model.predict(X_val).clip(0,600))
        pred = np.round(model.predict(X_test).clip(0,600))        
        out = pd.concat([out,pd.DataFrame(pred)],1)
        pred_ = pd.concat([pred_,pd.DataFrame(p)],1)
    
    pred_.columns = ['abw','pbw']
    
    print(f'mean_absolute_error:{mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))}')
    return out,pred_,mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))

In [368]:
out,pred,_ = training_loop_SVR()

[LibSVM]................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [379]:
out.columns = ['abw','pbw']
out.head()

,abw,pbw
0,4.0,1.0
1,0.0,21.0
2,10.0,0.0
3,5.0,0.0
4,0.0,1.0


In [395]:
out.head()

,abw,pbw
0,4.0,1.0
1,0.0,21.0
2,10.0,0.0
3,5.0,0.0
4,0.0,1.0


In [399]:
out.tail()

,abw,pbw
2798,0.0,7.0
2799,0.0,115.0
2800,0.0,0.0
2801,0.0,0.0
2802,0.0,7.0


In [400]:
# submit.tail()

submit[submit['image_id_worm']=='id_a93adfa0157cd5ac1ca7484f_pbw']

,image_id_worm,number_of_worms
3699,id_a93adfa0157cd5ac1ca7484f_pbw,NaN


In [401]:
submit.loc[::2,'number_of_worms']  = out.loc[:,'abw'].values
submit.loc[1::2,'number_of_worms'] = out.loc[:,'pbw'].values

In [402]:
submit.tail()

,image_id_worm,number_of_worms
5601,id_ffbcb27fa549278f47505515_pbw,0.0
5602,id_ffc0e41e10b0c964d4a02811_abw,0.0
5603,id_ffc0e41e10b0c964d4a02811_pbw,0.0
5604,id_fff8c253115aacded09ad7ed_abw,0.0
5605,id_fff8c253115aacded09ad7ed_pbw,7.0


In [403]:
submit[submit['image_id_worm']=='id_a93adfa0157cd5ac1ca7484f_pbw']

,image_id_worm,number_of_worms
3699,id_a93adfa0157cd5ac1ca7484f_pbw,0.0


In [404]:
submit.to_csv('/mnt/c/Personal/Competitions/Zindi/Wadhwani AI/runs/LGBM/V001/SVR.csv')

In [358]:
from sklearn.neural_network import MLPRegressor
def training_loop_nnet():
    
    out = pd.DataFrame()
    pred_ = pd.DataFrame()
   
    X_tr = consol_train_yolo.iloc[:,1:-2]
    Y_tr = consol_train_yolo.iloc[:,-2:]

    X_val = consol_val_yolo.iloc[:,1:-2]
    Y_val = consol_val_yolo.iloc[:,-2:]

    X_test = consol_test_yolo.iloc[:,1:]
        
    for i in range(2):
        model = MLPRegressor(hidden_layer_sizes= (2048,1024,512),activation='relu',verbose=True)
        model.fit(X_tr, Y_tr.iloc[:,i])
        p = np.round(model.predict(X_val).clip(0,600))
        pred = np.round(model.predict(X_test).clip(0,600))        
        out = pd.concat([out,pd.DataFrame(pred)],1)
        pred_ = pd.concat([pred_,pd.DataFrame(p)],1)
    
    pred_.columns = ['abw','pbw']
    
    print(f'mean_absolute_error:{mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))}')
    return out,pred_,mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))

In [359]:
training_loop_nnet()

Iteration 1, loss = 7.36451147
Iteration 2, loss = 0.42230672
Iteration 3, loss = 0.30056644
Iteration 4, loss = 0.27306559
Iteration 5, loss = 0.33533217
Iteration 6, loss = 0.24341067
Iteration 7, loss = 0.22818471
Iteration 8, loss = 0.27612861
Iteration 9, loss = 0.33530499
Iteration 10, loss = 0.22198595
Iteration 11, loss = 0.23367017
Iteration 12, loss = 0.18851148
Iteration 13, loss = 0.18369777
Iteration 14, loss = 0.19413770
Iteration 15, loss = 0.18253949
Iteration 16, loss = 0.17973349
Iteration 17, loss = 0.17614448
Iteration 18, loss = 0.22404191
Iteration 19, loss = 0.16461059
Iteration 20, loss = 0.16663404
Iteration 21, loss = 0.15527796
Iteration 22, loss = 0.15589590
Iteration 23, loss = 0.16042979
Iteration 24, loss = 0.20451619
Iteration 25, loss = 0.17233988
Iteration 26, loss = 0.15421119
Iteration 27, loss = 0.16275885
Iteration 28, loss = 0.15500499
Iteration 29, loss = 0.16541055
Iteration 30, loss = 0.15164687
Iteration 31, loss = 0.14680507
Iteration 32, los

(         0      0
 0      5.0    0.0
 1      0.0   19.0
 2     10.0    0.0
 3      4.0    0.0
 4      0.0    1.0
 ...    ...    ...
 2798   0.0    6.0
 2799   0.0  105.0
 2800   0.0    0.0
 2801   0.0    1.0
 2802   0.0    6.0
 
 [2803 rows x 2 columns],
        abw   pbw
 0      0.0   6.0
 1      0.0  71.0
 2      0.0   0.0
 3      0.0   6.0
 4      0.0   3.0
 ...    ...   ...
 1413  11.0   0.0
 1414   0.0  22.0
 1415   2.0   0.0
 1416   0.0   2.0
 1417   0.0  27.0
 
 [1418 rows x 2 columns],
 2.300423131170663)

In [363]:
from sklearn.ensemble import RandomForestRegressor
def training_loop_rf():
    
    out = pd.DataFrame()
    pred_ = pd.DataFrame()
   
    X_tr = consol_train_yolo.iloc[:,1:-2]
    Y_tr = consol_train_yolo.iloc[:,-2:]

    X_val = consol_val_yolo.iloc[:,1:-2]
    Y_val = consol_val_yolo.iloc[:,-2:]

    X_test = consol_test_yolo.iloc[:,1:]
        
    for i in range(2):
        model = RandomForestRegressor(n_jobs=-1,verbose=2)
        model.fit(X_tr, Y_tr.iloc[:,i])
        p = np.round(model.predict(X_val).clip(0,600))
        pred = np.round(model.predict(X_test).clip(0,600))        
        out = pd.concat([out,pd.DataFrame(pred)],1)
        pred_ = pd.concat([pred_,pd.DataFrame(p)],1)
    
    pred_.columns = ['abw','pbw']
    
    print(f'mean_absolute_error:{mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))}')
    return out,pred_,mean_absolute_error(Y_val.values.reshape(-1,1),pred_.values.reshape(-1,1))

In [364]:
training_loop_rf()

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    5.8s


building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   35.6s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.7s


building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   21.4s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 100 out of 100 | elapsed:    0.0s finished


(        0      0
 0     4.0    0.0
 1     0.0   20.0
 2     9.0    0.0
 3     5.0    0.0
 4     0.0    0.0
 ...   ...    ...
 2798  0.0    7.0
 2799  0.0  114.0
 2800  0.0    0.0
 2801  0.0    1.0
 2802  0.0    7.0
 
 [2803 rows x 2 columns],
        abw   pbw
 0      0.0   7.0
 1      0.0  72.0
 2      0.0   0.0
 3      0.0   7.0
 4      0.0   3.0
 ...    ...   ...
 1413  10.0   0.0
 1414   0.0  22.0
 1415   2.0   0.0
 1416   0.0   3.0
 1417   0.0  29.0
 
 [1418 rows x 2 columns],
 2.2789139633286317)

### Fin 